# V1 test

In [2]:
from trainer.trainer_basic import TSPN_trainer
from configs.config import args
from configs.config import signal_processing_modules,feature_extractor_modules
from pytorch_lightning.loggers import CSVLogger
import torch
import pytorch_lightning as pl

from data.data_provider import get_data
# dataset
train_dataloader, val_dataloader, test_dataloader = get_data(args)

name = f'model:{args.model}_lr:{args.learning_rate}_epochs:{args.num_epochs}_seed:{args.seed}_scale:{args.scale}'

print(f'Running experiment: {name}')
path = 'save/' + name

trainer = pl.Trainer(max_epochs=args.num_epochs,
                      devices= 1,
                      logger=CSVLogger(path, name = 'logs'),
                      log_every_n_steps=1,)

model = TSPN_trainer(signal_processing_modules, feature_extractor_modules, args)
best_model_path = 'save/lr0.001_epochs10_seed17_scale2_l1norm0.01/model-epoch=09-val_loss=0.4744.ckpt'
state_dict = torch.load(best_model_path)
model.load_state_dict(state_dict['state_dict'])

# 使用最佳模型进行测试
trainer.test(model, dataloaders=test_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


Running experiment: lr0.001_epochs10_seed17_scale2
# build signal processing layers
# build feature extractor layers
# build classifier
### network:
 Transparent_Signal_Processing_Network(
  (signal_processing_layers): ModuleList(
    (0): SignalProcessingLayer(
      (norm): InstanceNorm1d(2, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (weight_connection): Linear(in_features=2, out_features=6, bias=True)
      (signal_processing_modules): SignalProcessingModuleDict(
        (HT): HilbertTransform()
        (WF): WaveFilters()
        (I): Identity()
      )
      (skip_connection): Linear(in_features=2, out_features=6, bias=True)
    )
    (1-3): 3 x SignalProcessingLayer(
      (norm): InstanceNorm1d(6, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (weight_connection): Linear(in_features=6, out_features=6, bias=True)
      (signal_processing_modules): SignalProcessingModuleDict(
        (HT): HilbertTransform()
        (WF): WaveF

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      test_acc_epoch       │    0.4578651785850525     │
│      test_loss_epoch      │    1.2863938808441162     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss_epoch': 1.2863938808441162, 'test_acc_epoch': 0.4578651785850525}]

In [2]:
[1,2],[1,None]

([1, 2], [1, None])

# V2

In [3]:
############# learning ############
from cgi import test
from logging import config
from pytorch_lightning import seed_everything

from sklearn.calibration import log
import torch
############# config##########
import argparse
from model.TSPN import Transparent_Signal_Processing_Network
from trainer.trainer_basic import Basic_trainer
from trainer.trainer_set import trainer_set
# from configs.config import args
# from configs.config import signal_processing_modules,feature_extractor_modules
from configs.config import yaml_arguments,config_network
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '0' for test ##########################
import argparse


config_dir = 'configs/config_basic.yaml'


configs,args,path = yaml_arguments(config_dir)
seed_everything(args.seed)    
# 初始化模型
signal_processing_modules, feature_extractor_modules = config_network(configs,args)
network = Transparent_Signal_Processing_Network(signal_processing_modules, feature_extractor_modules,args)

trainer,train_dataloader, val_dataloader, test_dataloader = trainer_set(args,path)

model = Basic_trainer(network, args)

best_model_path = 'save/model_TSPN/time2024-03-12-19-26_lr0.001_epochs5_scale4_l1norm0.01_datasetTHU_018_basic_seed17/model-epoch=02-val_loss=1.7489.ckpt'
state_dict = torch.load(best_model_path)
model.load_state_dict(state_dict['state_dict'])

# 使用最佳模型进行测试
trainer.test(model, dataloaders=test_dataloader)

Seed set to 17


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Running experiment: time2024-03-12-19-56_lr0.001_epochs5_scale4_l1norm0.01_datasetTHU_018_basic_seed17
# build signal processing layers
# build feature extractor layers
# build classifier


You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: save/model_TSPN/time2024-03-12-19-56_lr0.001_epochs5_scale4_l1norm0.01_datasetTHU_018_basic_seed17/logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


### network:
 Transparent_Signal_Processing_Network(
  (signal_processing_layers): ModuleList(
    (0): SignalProcessingLayer(
      (norm): InstanceNorm1d(2, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (weight_connection): Linear(in_features=2, out_features=12, bias=True)
      (signal_processing_modules): SignalProcessingModuleDict(
        (HT): HilbertTransform()
        (WF): WaveFilters()
        (I): Identity()
      )
      (skip_connection): Linear(in_features=2, out_features=12, bias=True)
    )
    (1-3): 3 x SignalProcessingLayer(
      (norm): InstanceNorm1d(12, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (weight_connection): Linear(in_features=12, out_features=12, bias=True)
      (signal_processing_modules): SignalProcessingModuleDict(
        (HT): HilbertTransform()
        (WF): WaveFilters()
        (I): Identity()
      )
      (skip_connection): Linear(in_features=12, out_features=12, bias=True)
    )
  )
  (f

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.26140350103378296    │
│         test_loss         │    1.7489250898361206     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 1.7489250898361206, 'test_acc': 0.26140350103378296}]